# Mapeo campos 

|Fuente | Sources |
|---|---|
|Nombre base para flora de Bogota | Taxa |
|Comentario de validación | Taxa.Comment |
|Para revisión | Taxa.CheckPriority 0 or 1 |
|Endemica | Taxa.Distribution |
|Estado de amenaza | RedListAssessments with Standard='JBB'
|Origen y status | Origins |
|Trabajo de No nativas Flora de Bogotá | Origins.Type???
|Determinador en fuente | Identifications.IdentifiedBy Merge with next field |
|Determinador Depurado | Identifications.IdentifiedBy |
|Calificador en fuente | Identifications.Certainty |
|Fecha_Determinacion en fuente | Identifications.Date |
|Fecha_Determinacion_(Year) | Identifications.Date |
|Fecha_Determinacion_(Month) | Identifications.Date |
|Fecha_Determinacion_(Day) | Identifications.Date |
|State | Areas |
|County | Areas |
|Localidad | Locations |
|Código de localidad georreferenciada | Locations.BD2020ID |
|Latitud_original | Locations.Latitude |
|Longitud_original | Locations.Longitud |
|Elev__minima | Locations.ElevationMin |
|Elev__maxima | Locations.ElevationMax |
|Latitud_decimal | Geocodings.InterpretedLat, Geocoder is DB_2020 |
|Longitud_decimal | Geocodings.InterpretedLon, Geocoder is DB_2020 |
|Latitud para trabajo de mapas | Geocodings.InterpretedLat, Geocoder is mapas_2020 |
|Longitud para trabajo de mapas | Geocodings.InterpretedLon, Geocoder is mapas_2020 |
|Extensión de georreferenciación | Geocodins.Uncertainty ???
|DATUM | Geocodings.Datum |
|Colector depurado | Occurrences.Collector |
|Colector principal | If no colector depurado, Occurrences.Collector, otherwise Occurrences.CollectorVerbatim  |
|Número de colector | Occurrences.CollectionNumber & Occurrences.CollectionNumberVerbatim |
|Fecha_Inicial_(Year) | Occurrences.DateInit, Occurrences.DateEnd |
|Fecha_Inicial_(Month) | Occurrences.DateInit, Occurrences.DateEnd |
|Fecha_Inicial_(day) | Occurrences.DateInit, Occurrences.DateEnd |
|Notas_Localidad_y_Habitat | Occurrences.Comment  |
|Fenologia | Occurrences.PhenoState |
|Descripcion ejemplar | Occurrences.Comment |
|Usos | Occurrences.Use |
|Nombres_comunes | Occurrences.CommonName |
|Comentarios generales | Occurrences.Comment |
|Referencia bibligráfica | Occurrence.Sources |
|Numero original en Base 2015 | Occurrence.DB2015ID |
|Origen para herbario | Specimens.Institution  |
|Número de identificación | Specimens.SpecimenCode |
|Fecha_Inicial | REMOVE |
|Notas de altura | REMOVE ???????
|Familia Revisada | REMOVE |
|Género revisado | REMOVE |
|Comentarios de determinación en fuente | REMOVE |
|Nivel taxonómico | REMOVE, info retrieved from TPRS |
|Familia en fuente | REMOVE  |
|Género en fuente | REMOVE  |
|Epíteto específico en fuente | REMOVE  |
|Autor del epíteto específico en fuente | REMOVE  |
|Epíteto infraespecífico | REMOVE  |
|Autor del infraespecífico en fuente | REMOVE  |
|Nombre científico en fuente | REMOVE  |
|Nombre validado | REMOVE  |
|Nivel taxonómico en fuente | REMOVE  |
|Herbario de proveniencia | REMOVE  |

In [194]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import requests
import json
import time
import pykew.powo as powo
import re

import credentials

In [162]:
def tropicos(name, api_key):
	service = 'http://services.tropicos.org/name/search'
	r = requests.get(
		f'{service}?apikey={api_key}&name={name}&format=json'
	)
	return r.json()


In [195]:
def powo_query(name):
	act_name = ''
	bits = re.split(r'\s+', name)

	if bits[0] == '×' or bits[1] == '×':
		act_name = ' '.join(bits[:3])
	else:
		act_name = ' '.join(bits[:2])
	
	r = powo.search(act_name)

	if r.size() == 1:
		results = [i for i in r]
		return (results[0]['name'], results[0]['author'])
	
	elif r.size() > 1:
		return f'Multiples results with name {name}'
	
	else:
		return f'No results with name {name}'

In [ ]:
# Map between original name and corrected version by TNRS
taxa_map = None

In [3]:
orifile = '../Datos/Flora_de_Bogota/Registros_2020/FlBogota2020.csv'
ori = pd.read_csv(orifile, low_memory=False)

In [8]:
tnrs_url = 'https://tnrsapi.xyz/tnrs_api.php'

In [26]:
orinames = ori['Nombre base para flora de Bogota'].unique()

In [104]:
query = [[i, j] for i,j in enumerate(orinames) if pd.notna(j)]

In [110]:
r = requests.post(
	tnrs_url, 
	headers={"Content-Type": "application/json"},
	data=json.dumps({
		'data': query,
		'opts': {
			'sources': ['wcvp','wfo'],
			'class': 'wcvp',
			'mode': 'resolve',
			'matches' : 'best'
		}		
	})	
)

In [111]:
results = r.json()

In [113]:
len(results)

3830

In [118]:
res = pd.DataFrame.from_records(results)

In [120]:
res.to_csv('tnrs_query.csv', index=False)

In [128]:
res.loc[res.Overall_score == '', 'Overall_score'] = 'NAN'

In [130]:
res['Overall_score'] = res.Overall_score.astype(np.float32)

In [132]:
res.loc[res.Overall_score.isna()]

,ID,Name_submitted,Overall_score,Name_matched_id,Name_matched,Name_score,Name_matched_rank,Author_submitted,Author_matched,Author_score,...,Accepted_name_id,Accepted_name_rank,Accepted_name_url,Accepted_name_lsid,Accepted_family,Overall_score_order,Highertaxa_score_order,Source,Warnings,WarningsEng
1383,1383,Pteridophyta,NaN,,[No match found],,,,,,...,,,,,,,,,,
1715,1715,Magnoliophyta,NaN,,[No match found],,,,,,...,,,,,,,,,,
2804,2804,Candelariella,NaN,,[No match found],,,,,,...,,,,,,,,,,
3316,3317,Tracheophyta,NaN,,[No match found],,,,,,...,,,,,,,,,,


In [177]:
res.loc[res.Overall_score < 0.8, ['Name_submitted', 'Name_matched']]

,Name_submitted,Name_matched
305,Achyrocline elegans,Achyrocline
385,Arenaria bogotensis,Arenaria
392,Arthrostylidium latifolia,Arthrostylidium
400,Asplenium micranthum,Asplenium
448,Bomarea guadalupe,Bomarea
477,Bulbostylis tropicalis,Bulbostylis
484,Calceolaria arvensis,Calceolaria
486,Calceolaria fisifolia,Calceolaria
518,Carex peuciphalia,Carex
541,Ceroxylon andinus,Ceroxylon


In [191]:
powo_query('Ageratina asclepiadea × gynoxoides')

'No results with name Ageratina asclepiadea × gynoxoides'

In [163]:
tropicos('Crinum × powellii', credentials.tropicos['api_key'])

[{'NameId': 1201645,
  'ScientificName': 'Crinum × powellii',
  'ScientificNameWithAuthors': 'Crinum × powellii hort. ex Baker',
  'Family': 'Amaryllidaceae',
  'RankAbbreviation': 'nothosp.',
  'NomenclatureStatusName': 'No opinion',
  'Author': 'hort. ex Baker',
  'DisplayReference': 'Handb. Amaryll. 95',
  'DisplayDate': '1888',
  'TotalRows': 1}]

In [151]:
r = requests.post(
	'http://services.tropicos.org/name/search',
	data=json.dumps({
		'apikey': credentials.tropicos['api_key'],
		'name': 'Aragoa',
		'format': 'json'
	})
)

In [178]:
res.loc[res.Name_submitted.str.contains('×'), ['Name_submitted', 'Name_matched', 
	'Name_matched_rank', 'Accepted_name', 'Accepted_name_rank']]

,Name_submitted,Name_matched,Name_matched_rank,Accepted_name,Accepted_name_rank
328,Ageratina asclepiadea × gynoxoides,Ageratina asclepiadea,species,Ageratina asclepiadea,species
376,Aragoa × funzana Fern.Alonso,Aragoa funzana,species,Aragoa x funzana,species
611,Crinum × powellii Baker,Crinum powellii,species,,
614,Crocosmia × crocosmiiflora (Lemoine) N.E.Br.,Crocosmia crocosmiiflora,species,Crocosmia x crocosmiiflora,species
848,Glandularia × hortulanus,Glandularia,genus,Verbena,genus
1091,Mentha × piperita L.,Mentha piperita,species,Mentha x piperita,species
1237,Passiflora × rosea (H.Karst.) Killip,Passiflora rosea,species,Passiflora x rosea,species
1759,Brugmansia × candida Pers.,Brugmansia candida,species,Brugmansia x candida,species
1823,Salvia × rociana Fern.Alonso,Salvia rociana,species,Salvia x rociana,species
1974,Abutilon × hybridum hort. ex Voss,Abutilon hybridum,species,,


In [196]:
res.loc[res.Name_submitted.str.contains('×'), 'Name_submitted'].apply(powo_query)

328     (Ageratina asclepiadea, (L.f.) R.M.King & H.Rob.)
376                       (Aragoa × funzana, Fern.Alonso)
611                            (Crinum × powellii, Baker)
614       (Crocosmia × crocosmiiflora, (Lemoine) N.E.Br.)
848         No results with name Glandularia × hortulanus
1091                              (Mentha × piperita, L.)
1237              (Passiflora × rosea, (H.Karst.) Killip)
1759                        (Brugmansia × candida, Pers.)
1823                      (Salvia × rociana, Fern.Alonso)
1974                          (Abutilon × hybridum, Voss)
2120                   (Aragoa × jaramilloi, Fern.Alonso)
2150    Multiples results with name Pelargonium × hort...
2394                     (Espeletia × verdeana, Cuatrec.)
2678                (Salvia × tunica-mariae, Fern.Alonso)
2680                   (Salvia gachantivana, Fern.Alonso)
2681                   (Salvia gachantivana, Fern.Alonso)
2682                   (Salvia gachantivana, Fern.Alonso)
2683          

In [20]:
for myr in results:
	for k in myr.keys():
		print(k)
	break

ID
Name_submitted
Overall_score
Name_matched_id
Name_matched
Name_score
Name_matched_rank
Author_submitted
Author_matched
Author_score
Canonical_author
Name_matched_accepted_family
Genus_submitted
Genus_matched
Genus_score
Specific_epithet_submitted
Specific_epithet_matched
Specific_epithet_score
Family_submitted
Family_matched
Family_score
Infraspecific_rank
Infraspecific_epithet_matched
Infraspecific_epithet_score
Infraspecific_rank_2
Infraspecific_epithet_2_matched
Infraspecific_epithet_2_score
Annotations
Unmatched_terms
Name_matched_url
Name_matched_lsid
Phonetic
Taxonomic_status
Accepted_name
Accepted_species
Accepted_name_author
Accepted_name_id
Accepted_name_rank
Accepted_name_url
Accepted_name_lsid
Accepted_family
Overall_score_order
Highertaxa_score_order
Source
Warnings
WarningsEng


In [ ]:
conn_str = 'mysql+mysqlconnector://' + \
	f'{credentials.mysql['username']}:{credentials.mysql['password']}' + \
	'@localhost:3306/Mutis'
engine = create_engine(conn_str)
connection = engine.connect()

In [7]:
df = pd.read_sql(sql='SELECT * FROM Taxa', con=connection)

In [8]:
df

,TaxonID,Name,AcceptedName,Authority,Author,Protologue,Parent,Comment,CheckPriority,Distribution,TimeStamp


In [9]:
connection.close()